<img src="../img/pandora2d_logo.png" width="500" height="500">

# Criteria

The validity of each pixel and its associated similarity values is flagged so that there is an information about what a computation can do with a pixel.

This Notebook propose to see how those flags (called `Criteria` in Pandora2D) are used.

## Basics of Criteria

`Criteria` is an Enum located in the `constants` module of pandora2d:

In [ ]:
from pandora2d.constants import Criteria

`Criteria` gots members who's values are power of 2. It's possible to list them by iterating the `Criteria` class:

In [ ]:
list(Criteria)

`Criteria` inherits from `enum.IntFlag`. One can refer to the [official documentation](https://docs.python.org/3/howto/enum.html#intflag) to get more information on its usage, but basically, here are the two most common usages:

- `Criteria` values can be combined with the logical or operator:

In [ ]:
new_criteria = Criteria.P2D_INVALID_MASK_RIGHT | Criteria.P2D_PEAK_ON_EDGE
new_criteria

- one can check if a `Criteria` value is part of a compound criteria with the `in` statement:

In [ ]:
Criteria.P2D_PEAK_ON_EDGE in new_criteria

## Usage with numpy arrays

Internally values are stored as integers. So when `Criteria` are stored in a numpy array, its dtype is `np.int64`:

In [ ]:
import numpy as np

In [ ]:
array = np.array(
    [
        Criteria.P2D_INVALID_MASK_LEFT,
        Criteria.P2D_PEAK_ON_EDGE | Criteria.P2D_INVALID_MASK_RIGHT,
    ]
)

In [ ]:
array.dtype

It's possible to force the dtype to be `np.uint8` in order to take less space in memory:

In [ ]:
array_uint8 = np.array(
    [
        Criteria.P2D_INVALID_MASK_LEFT,
        Criteria.P2D_PEAK_ON_EDGE | Criteria.P2D_INVALID_MASK_RIGHT,
    ],
    dtype=np.uint8,
)

In [ ]:
array_uint8.dtype

In [ ]:
print(f"Size of array in memory: {array.nbytes} bytes\nSize of the array_uint8 in memory: {array_uint8.nbytes} bytes")

In both cases, arrays store integers, losing the relationship with `Criteria` objects, so the array representation is not meaningful:

In [ ]:
array_uint8

Moreover, the `in` statement can not be used anymore to check if a `Criteria` is part of a value.
This will fail:

```python
Criteria.P2D_INVALID_MASK_RIGHT in array_uint8[0]
```

The value must be converted back to `Criteria` first:

In [ ]:
first_criteria = Criteria(array_uint8[0])
first_criteria

In [ ]:
Criteria.P2D_INVALID_MASK_RIGHT in first_criteria

### `is_in` method

As it is not very convenient to do this conversion each time, especially if it is needed to check a whole array, the method `is_in` was added to `Criteria`.

It can take a single value as an argument:

In [ ]:
Criteria.P2D_INVALID_MASK_RIGHT.is_in(array_uint8[0])

Or an array:

In [ ]:
Criteria.P2D_INVALID_MASK_RIGHT.is_in(array_uint8)

### FlagArray

Futhermore, `Criteria` are stored in a dedicated numpy array subtype located in the pandora2d criteria module:

In [ ]:
from pandora2d.criteria import FlagArray

This is an ordinary array that stores `Criteria` as `np.uint8`, but an additional parameter that takes `Criteria` as argument was added:

In [ ]:
flag_array = FlagArray(
    [
        Criteria.P2D_INVALID_MASK_LEFT,
        Criteria.P2D_PEAK_ON_EDGE | Criteria.P2D_INVALID_MASK_RIGHT,
    ],
    flag=Criteria,
)

This additional parameter is used to change the representation of the array:

In [ ]:
flag_array

but values are still `np.uint8`:

In [ ]:
type(flag_array[0])

so, to extract a `Criteria` from the array, the value still needs to be converted back to `Criteria`:

In [ ]:
Criteria(flag_array[0])

### Implicit convertion

<div class="alert alert-block alert-warning"> 
 Beware implicite convertion when operation between Criteria and numpy arrays are done!
 </div>

Operation between `Criteria` and numpy array does an implicit convertion to `int64`:

In [ ]:
flag_array | Criteria.P2D_RIGHT_DISPARITY_OUTSIDE

When doing an in place operation, it will not be allowed without doing an explicit cast to `np.uint8`!

Do not do:

```python
flag_array |= Criteria.P2D_RIGHT_DISPARITY_OUTSIDE
```

Do:

In [ ]:
flag_array |= np.uint8(Criteria.P2D_RIGHT_DISPARITY_OUTSIDE)

flag_array

### The VALID flag

The `FlagArray` used in Pandora2D is initialized with the special `Criteria.VALID` flag which value is `0`.

In [ ]:
Criteria.VALID

When it is combined with another flag, this one takes the predominance:

In [ ]:
Criteria.VALID | Criteria.P2D_LEFT_BORDER

Indeed, if a flag is raised, the pixel is not *valid* anymore!

<div class="alert alert-block alert-warning"> 
 But cares must be taken when the validity of a `Critera` has to be checked!
 </div>

Since it's value is `0`, it is found in every other `Criteria`:

In [ ]:
Criteria.VALID.is_in(np.array(Criteria))

In [ ]:
valid_array = np.array([Criteria.VALID, Criteria.P2D_PEAK_ON_EDGE], dtype=np.uint8)

In [ ]:
Criteria.VALID == valid_array

While it is possible to use this method, it is more efficient to use the fact that `VALID` value is `0` and that `0` is falsy (i.e. when converted to a bool, it is considered as `False` while other values are considered `True`):

In [ ]:
~valid_array.astype(bool)

In [ ]:
big_valid_array = np.full(10000, Criteria.VALID, dtype=np.uint8)
big_valid_array[::2] = np.uint8(Criteria.P2D_PEAK_ON_EDGE)
big_valid_array

In [ ]:
%%timeit
Criteria.VALID == big_valid_array

In [ ]:
%%timeit
~big_valid_array.astype(bool)